In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import MultinomialNB
import tkinter as tk
from tkinter import ttk
from PIL import ImageTk, Image
import requests
from io import BytesIO

# Data collection
df = pd.read_csv('anime_list.csv')

# Data preprocessing
df = df.drop_duplicates(subset='title', keep='first')
df = df.dropna(subset=['synopsis'])

# Feature extraction with CountVectorizer
cv = CountVectorizer(stop_words='english')
synopsis_matrix = cv.fit_transform(df['synopsis'])
cosine_sim_count = cosine_similarity(synopsis_matrix)

# Feature extraction with TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
synopsis_matrix_tfidf = tfidf.fit_transform(df['synopsis'])
cosine_sim_tfidf = cosine_similarity(synopsis_matrix_tfidf)

# K-Nearest Neighbors model training with CountVectorizer
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(synopsis_matrix)

# Random Forest model training with CountVectorizer
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(synopsis_matrix, df['score'])

# Naive Bayes model training with CountVectorizer
nb = MultinomialNB()
nb.fit(synopsis_matrix, df['title'])

# Function to get recommendation using cosine similarity with CountVectorizer
def recommend_anime_count(title):
    idx = df[df['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim_count[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[anime_indices]

# Function to get recommendation using cosine similarity with TfidfVectorizer
def recommend_anime_tfidf(title):
    idx = df[df['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    anime_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[anime_indices]

# Function to get KNN recommendations
def recommend_anime_knn(title):
    idx = df[df['title'] == title].index[0]
    distances, indices = knn.kneighbors(synopsis_matrix[idx], n_neighbors=11)
    anime_indices = indices[0][1:]
    return df['title'].iloc[anime_indices]

# Function to get Random Forest recommendations
def recommend_anime_rf(title):
    idx = df[df['title'] == title].index[0]
    synopsis = synopsis_matrix_tfidf[idx].toarray()
    for i in range(10):
        distances, indices = knn.kneighbors(synopsis, n_neighbors=11)
        anime_indices = indices[0][1:]
        synopsis = np.concatenate((synopsis, synopsis_matrix_tfidf[anime_indices].toarray()), axis=0)
    scores = rf.predict(synopsis)
    anime_indices = np.argsort(scores)[::-1][:10]
    return df['title'].iloc[anime_indices]

# Function to get Naive Bayes recommendations
def recommend_anime_nb(title):
    idx = df[df['title'] == title].index[0]
    synopsis = synopsis_matrix_tfidf[idx].toarray()
    for i in range(10):
        distances, indices = knn.kneighbors(synopsis, n_neighbors=11)
        anime_indices = indices[0][1:]
        synopsis = np.concatenate((synopsis, synopsis_matrix_tfidf[anime_indices].toarray()), axis=0)
    scores = nb.predict_proba(synopsis)
    anime_indices = np.argsort(scores[:, 1])[::-1][:10]
    return df['title'].iloc[anime_indices]

# GUI setup
root = tk.Tk()
root.title('Anime Recommender')

# Add wallpaper
response = requests.get('https://th.bing.com/th/id/R.9507efa52b14e038eefc85ba5d74e448?rik=As6NiN0V3HtpZg&pid=ImgRaw&r=0')
wallpaper = Image.open(BytesIO(response.content))
wallpaper = wallpaper.resize((1920, 1080), Image.ANTIALIAS)
wallpaper = ImageTk.PhotoImage(wallpaper)
background_label = tk.Label(root, image=wallpaper)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Label for title
title_label = tk.Label(root, text='Anime Recommender', font=('Arial', 24))
title_label.pack(pady=10)

# Dropdown menu for selecting recommendation model
model_label = tk.Label(root, text='Select a recommendation model:', font=('Arial', 14))
model_label.pack()
model_var = tk.StringVar()
model_dropdown = ttk.Combobox(root, textvariable=model_var, font=('Arial', 14), state='readonly')
model_dropdown['values'] = ('Cosine Similarity with CountVectorizer', 'Cosine Similarity with TfidfVectorizer', 
                            'K-Nearest Neighbors with CountVectorizer', 'Random Forest with CountVectorizer', 
                            'Naive Bayes with CountVectorizer')
model_dropdown.pack(pady=10)

# Label and entry for input title
input_label = tk.Label(root, text='Enter an anime title:', font=('Arial', 14))
input_label.pack()
input_var = tk.StringVar()
input_entry = tk.Entry(root, textvariable=input_var, font=('Arial', 14))
input_entry.pack(pady=10)

# Function to get recommendations based on selected model and input title
def get_recommendations():
    title = input_var.get()
    model = model_var.get()
    if model == 'Cosine Similarity with CountVectorizer':
        recommendations = recommend_anime_count(title)
    elif model == 'Cosine Similarity with TfidfVectorizer':
        recommendations = recommend_anime_tfidf(title)
    elif model == 'K-Nearest Neighbors with CountVectorizer':
        recommendations = recommend_anime_knn(title)
    elif model == 'Random Forest with CountVectorizer':
        recommendations = recommend_anime_rf(title)
    elif model == 'Naive Bayes with CountVectorizer':
        recommendations = recommend_anime_nb(title)
    output_text.delete('1.0', tk.END)
    output_text.insert(tk.END, recommendations.to_string(index=False))

# Button to get recommendations
button = tk.Button(root, text='Get Recommendations', font=('Arial', 14), command=get_recommendations)
button.pack(pady=10)

# Output text area for recommendations
output_frame = tk.Frame(root, bg='black')
output_frame.pack(pady=10)
output_label = tk.Label(output_frame, text='Recommendations:', font=('Arial', 14), fg='white', bg='black')
output_label.pack(pady=5)
output_text = tk.Text(output_frame, font=('Arial', 14))
output_text.pack(pady=5)
output_text.config(width=40, height=11)

root.mainloop()

C:\Users\asus\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\asus\AppData\Local\Temp\ipykernel_15580\1986867300.py:100: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  wallpaper = wallpaper.resize((1920, 1080), Image.ANTIALIAS)
